## Visualize collected `geospatial data` on a <span style="color:turquoise;">MAP</span>

In [36]:
import requests
#Define the bldg code service function
#ONU
#Enter user number
num = int(input("Please enter a number: "))
def getONU(num):
    url = f"http://105.29.165.253:13000/api/komp_assistant/onu/phone/{num}"

    payload = {}
    headers = {
    'Authorization': 'Bearer ***REMOVED***'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code == 200:
        data = response.json()
        bcode_value = data.get("bcode")
        bname_value = data.get("bname")
        return bcode_value, bname_value
    else:
        bcode_value = None
        return None

def getPAP(num):
    url = f"http://105.29.165.253:13000/api/komp_assistant/phone/{num}"

    payload = {}
    headers = {
    'Authorization': 'Bearer ***REMOVED***'
    }

    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code == 200:
        data = response.json()
        bcode_value = data.get("bcode")
        bname_value = data.get("bname")
        return bcode_value, bname_value
    else:
        bcode_value = None
        return None

def getbcode(num):
    try:
        bcode, bname = getPAP(num)
        if bcode == None:
            bcode = getONU(num)
            if bcode == None:
                print('Bcode not found for both PAP and ONU')
                return 'Not Found'
            else:
                print('ONU', num, bcode)
                return 'ONU', num, bcode, bname
        else:
            print('PAP', num, bcode)
            return 'PAP', num, bcode, bname
    except Exception as e:
        print('Error: ',e)

result = getbcode(num)
print(result)
if isinstance(result, tuple):
    print("Result is a tuple")
    router, num, buildingcode, bname = result
else:
    print("Result is not a tuple")
    buildingcode = 'Not Exist'


PAP 706139071 KENRZ18020
('PAP', 706139071, 'KENRZ18020', 'Harvest')
Result is a tuple


In [37]:
#%pip install folium
import zipfile
import xml.etree.ElementTree as ET
import folium

with zipfile.ZipFile('./geobank/KCIS Master Map 3010_2023.kmz', 'r') as zip_ref:
    zip_ref.extractall('./geobank/')

# Parse the .kml file
tree = ET.parse('./geobank/doc.kml')
root = tree.getroot()

In [38]:
for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
    name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
    #print(name)

In [39]:
print(buildingcode)
buildingcode = str(buildingcode)
for placemark in root.iter('{http://www.opengis.net/kml/2.2}Placemark'):
    name = placemark.find('{http://www.opengis.net/kml/2.2}name').text
    #print(name)
    if buildingcode in name: 
        coordinates = placemark.find('{http://www.opengis.net/kml/2.2}Point/{http://www.opengis.net/kml/2.2}coordinates').text
        # coordinates will be a string in the format "longitude,latitude,altitude"
        # you can split it into a list of floats using:
        coordinates_list = [float(coord) for coord in coordinates.split(',')]
        print(coordinates_list)

KENRZ18020
[36.89624276060896, -1.206767622731932, 0.0]


In [35]:
longitude, latitude = coordinates_list[:2]
# Create a folium map centered around the coordinates
map_object = folium.Map(location=[latitude, longitude], zoom_start=15)

# Add a marker to the map
folium.Marker(location=[latitude, longitude], popup=f'Contact: {num}, Router Type: {router}, Building: {bname}').add_to(map_object)

# Save the map to an HTML file
map_object.save('./geobank/map.html')